# GHOSTS

Creates a melody with a time-stretched accompanyment of high tones, low thumps, and strums that spread out over longer and longer timepoints in the future.

<hr style="height:1px;color:gray">

Notebook imports:

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from musx import version, setmidiplayer, playfile, Score, Seq, Note, MidiFile, between, pick, steps
from musx.midi.gm import Flute, Clarinet, Cello, OrchestralHarp
print(f'musx version:', version)

This notebook generates MIDI files and automatically plays them using [fluidsynth](https://www.fluidsynth.org/download/) and the [MuseScore_General.sf3](https://ftp.osuosl.org/pub/musescore/soundfont/MuseScore_General) sound font. See [INSTALL.md](https://github.com/musx-admin/musx/blob/main/INSTALL.md) for how to install a terminal-based MIDI player to use with musx.  If you don't have a player installed you can access the output files in the same directory as this notebook:

In [ ]:
setmidiplayer("fluidsynth -iq -g1 /usr/local/sf/MuseScore_General.sf3")
print('OK!')

The flute generator creates a high, temporally stretched version of the main melody played by the clarinet:

In [ ]:
def flute(score, knum, dur):
    """
    Creates the flute part on channel 0.
    
    Parameters
    ----------
    score : Score
        The musical score to add events to.
    knum : int
        The midi keynumber of the clarinet.
    dur : int | float
        The duration of the note.
    """
    score.add(Note(time=score.now, duration=dur, pitch=knum+24, amplitude=.2, instrument=0))
    # yielding -1 stops this generator from running again score's processing queue
    yield -1

print(f"flute: {flute}")

The cello plays a low, percussive "thump" to accompany low melodic tones in the clarinet:

In [ ]:
def cello(score, knum):
    """
    Creates the cello part on channel 2.
    
    Parameters
    ----------
    score : Score
        The musical score to add events to.
    knum : int
        The midi key number of the clarinet.
    """
    score.add(Note(time=score.now, duration=.05, pitch=knum-18, amplitude=.9, instrument=2))
    score.add(Note(time=score.now, duration=.05, pitch=knum-23, amplitude=.9, instrument=2))
    # yielding -1 stops this generator from running again score's processing queue
    yield -1

print(f"cello: {cello}")

Adds a distant strum that accompanies the high stretched melody in the flute but at even larger time scales:

In [ ]:
def harp(score, knum, rate):
    """
    Creates an arpeggiating harp part on channel 3.
    
    Parameters
    ----------
    score : Score
        The musical score to add events to.
    knum : int
        The midi keynumber of the clarinet.
    rate : int | float
        The rhythm of the arpeggio.
    """
    for k in steps(39 + (knum % 13),  13, 5):
        m = Note(time=score.now, duration=10, pitch=k, amplitude=.5, instrument=3)
        score.add(m)
        yield rate

print(f"harp: {harp}")

The ghosts process. Generates 12 notes and adds accompanying figures in the cello and flute depending qualities of the main melody:

In [ ]:
def ghosts(score):
    """
    Creates mid-range clarinet line and decorates it with
    calls to the other instrument composers.
    
    Parameters
    ----------
    score : Score
        The musical score to add events to.
    """
    for _ in range(12):
        here = score.elapsed
        ahead = (here + 1/2) * 2
        melody = between(53, 77)
        high = (melody >=  65)
        amp = .2 if high else .4
        rhy = pick(1/4, 1/2, 3/4)
        # the clarinet line
        note = Note(time=score.now, duration=rhy + .2, pitch=melody, amplitude=amp, instrument=1)
        score.add(note)
        # add decorations to the clarinet melody
        if high:
            score.compose([ahead, flute(score, melody, ahead)])
            score.compose([ahead * 2, harp(score, melody, rhy / 4)])
        elif (rhy == 3/4):
            score.compose([1/2, cello(score, melody)])
        yield rhy

print(f"ghosts: {ghosts}")

Define track0 to be a midi meta track that holds tempo, midi instrument assignments, micro tuning, etc.:

In [ ]:
track0 = MidiFile.metatrack(ins={0: Flute, 1: Clarinet, 2: Cello, 3: OrchestralHarp})
print(f"track0: {track0}")

Track1 will hold the composition:

In [ ]:
track1 = Seq()
print(f"track1: {track1}")

Create a score and pass it track1 to receive the output midi event data:

In [ ]:
score = Score(out=track1)
print(f"score: {score}")

Create the composition:

In [ ]:
score.compose(ghosts(score))
print("OK!")

Write the tracks to a midi file in the current directory and play it if possible:

In [ ]:
file = MidiFile("ghosts.mid", [track0, track1]).write()
print(f"Wrote '{file.pathname}'")
playfile(file.pathname)